# Team 4 Topic: Suicide

# How we went about getting the Data.:
## We wanted to explore the topic about mortality and after exploring a little we bit we found the topic of Suicide to be worth delving deeper in. 

# Where did we find the Data?
## We ended up finding 3 source location in which we were able to find our main data source plus additional supporting documents to make it easier to interpret the data. Such as documents we used to decypher the code.
# Sources: 
## Kaggle "https://www.kaggle.com/cdc/mortality"
## Main Source
## http://www.cms.hhs.gov/ICD10/downloads/Dxgem_2009.zip"
## Health Code txt files
## "https://www.cdc.gov/nchs/data/dvs/Record_Layout_2010.pdf "
## Quick Code Reference


# Project 1: Suicide
## Part 1: Data Cleanup
The data came in a zip folder from <a href="https://www.kaggle.com/cdc/mortality">kaggle.</a>  There were 11 CSVs that were millions of rows by 77 columns.  To get it into a format that can be worked with, a batch at a time were read into multiple dataframes, those dataframes were appended to each other, a sample was taken, and then the process was repeated with a second batch of CSVs.  The two sample CSVs were then appended to each other.  The reason for doing this in sections was because the dataframes were so large they were eating up too much memory.  The next three code blocks show that process.
## Code Cleanup Block 1
Reading years 2005-2009 into dataframes, taking a sample, and appending them to one.

In [ ]:
# This is the code for originally creating the dataset from
# 11 different CSVs.  All other code cleaning is in my 
# main data exploration notebook.

# There are three code blocks here
# Run one and then clear the kernal and run the next
import pandas as pd

# read CSVs into DataFrame
df2005 = pd.read_csv("2005_data.csv", low_memory=False)
df2006 = pd.read_csv("2006_data.csv", low_memory=False)
df2007 = pd.read_csv("2007_data.csv", low_memory=False)
df2008 = pd.read_csv("2008_data.csv", low_memory=False)
df2009 = pd.read_csv("2009_data.csv", low_memory=False)

# append all DataFrames to one
death05_09 = df2005.append([df2006, df2007, df2008, df2009])

# keep only necessary columns
death05_09 = death05_09[['resident_status','education_2003_revision',
                          'month_of_death','sex','detail_age_type',
                          'detail_age','age_recode_52','age_recode_27',
                          'age_recode_12','infant_age_recode_22',
                          'place_of_death_and_decedents_status',
                          'marital_status','day_of_week_of_death',
                          'current_data_year','injury_at_work','manner_of_death',
                          'method_of_disposition','autopsy','activity_code',
                          'icd_code_10th_revision','358_cause_recode','113_cause_recode',
                          '130_infant_cause_recode','39_cause_recode','race_recode_3',
                          'race_recode_5','hispanic_origin','hispanic_originrace_recode']]
# take a sample
death05_09sample = death05_09.sample(frac=0.05, replace=True, random_state=1)

# save to CSV
death05_09sample.to_csv("death05_09sample.csv")

## Code Cleanup Block 2
Reading years 2010-2015 into dataframes, taking a sample, and appending them to one.

In [ ]:
# Make sure to clear kernal before running this block
import pandas as pd

# read CSVs into DataFrame
df2010 = pd.read_csv("2010_data.csv", low_memory=False)
df2011 = pd.read_csv("2011_data.csv", low_memory=False)
df2012 = pd.read_csv("2012_data.csv", low_memory=False)
df2013 = pd.read_csv("2013_data.csv", low_memory=False)
df2014 = pd.read_csv("2014_data.csv", low_memory=False)
df2015 = pd.read_csv("2015_data.csv", low_memory=False)

# append all DataFrames to one
death10_15 = df2010.append([df2011, df2012, df2013, df2014, df2015])

# keep only necessary columns
death10_15 = death10_15[['resident_status','education_2003_revision',
                          'month_of_death','sex','detail_age_type',
                          'detail_age','age_recode_52','age_recode_27',
                          'age_recode_12','infant_age_recode_22',
                          'place_of_death_and_decedents_status',
                          'marital_status','day_of_week_of_death',
                          'current_data_year','injury_at_work','manner_of_death',
                          'method_of_disposition','autopsy','activity_code',
                          'icd_code_10th_revision','358_cause_recode','113_cause_recode',
                          '130_infant_cause_recode','39_cause_recode','race_recode_3',
                          'race_recode_5','hispanic_origin','hispanic_originrace_recode']]
# take a sample
death10_15sample = death05_09.sample(frac=0.05, replace=True, random_state=1)

# save to CSV
death10_15sample.to_csv("death10_15sample.csv")

## Code Cleanup Block 3
Appending the two sample CSVs to one.

In [ ]:
# Make sure to clear kernal before running this block
import pandas as pd

# read CSVs into DataFrame
df10_15 = pd.read_csv("death10_15sample.csv")
df05_09 = pd.read_csv("death05_09sample.csv")

# append all DataFrames to one
sample5death05_15 = df10_15.append(df05_09)

# save to CSV
sample5death05_15.to_csv("sample5death05_15.csv")

## Part 2: Formatting
After the data was put into a usable format it was necessary to expand on that work further.  The code block below is the start of how the data was analyzed everytime our project was returned to.  The process goes:  
* Import all libraries (used and experimented with)
* Read the sample CSV into the dataframe
* Create functions that will be applied to columns to create new defining columns
* Read JSON files into dataframes so that they can be appended to the main sample dataframe
* Create dummy columns to be used in data aggregation and manipulation  

There were a few ages that were recorded incorrectly.  I dropped rows where the age was above 150.  I googled the oldest person to live and it was 122. So, just to be safe I dropped anything under 150.  But the ages that were wrong were like 999, 888, 777, etc.

In [ ]:
#Dependencies
#%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
deathdf = pd.read_csv("sample5death05_15.csv", low_memory=False)

In [ ]:
import pandas as pd
from pprint import pprint as p
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import joypy
from matplotlib import cm
import warnings
warnings.simplefilter("ignore")

# import the dataset
deathdf = pd.read_csv("sample5death05_15.csv", low_memory=False)

# functions to create helper columns


def suiYN(x):
    if x in (124, 125, 126):
        return "Suicide"
    else:
        return "Not Suicide"

def mthd(x):
    if x == 125:
        return "Firearm"
    else:
        return "Other Means"
    
def suCnt(x):
    if x == "Suicide":
        return 1
    else:
        return 0

def othrCnt(x):
    if x == "Not Suicide":
        return 1
    else:
        return 0
    
def firearmCnt(x):
    if x == "Firearm":
        return 1
    else:
        return 0

def nonFirearmCnt(x):
    if x == "Other Means":
        return 1
    else:
        return 0
    
# helper columns
deathdf["SuicideYN"] = deathdf["113_cause_recode"].apply(suiYN)
deathdf["SuicideCnt"] = deathdf["SuicideYN"].apply(suCnt)
deathdf["OtherCnt"] = deathdf["SuicideYN"].apply(othrCnt)
deathdf["Method"] = deathdf["113_cause_recode"].apply(mthd)
deathdf["FirearmCnt"] = deathdf["Method"].apply(firearmCnt)
deathdf["NonFirearmCnt"] = deathdf["Method"].apply(nonFirearmCnt)
    
# import json file
jsonfile = "2015_codes.json"
json_data=open(jsonfile).read()
data = json.loads(json_data)

# retrieve educational data
eddf = pd.DataFrame.from_dict(data['education_2003_revision'], orient="index")
eddf = eddf.reset_index()
eddf = eddf.rename(columns={0:"Education",
                           "index": "EdCode"})
eddf["EdCode"] = eddf["EdCode"].astype(float)

# retrieve marital status data
mardf = pd.DataFrame.from_dict(data["marital_status"], orient="index")
mardf = mardf.reset_index()
mardf = mardf.rename(columns={0 : "Marital Status",
                             "index": "MarCode"})
# merge them to the deathdf
deathdf = deathdf.merge(eddf, left_on="education_2003_revision", right_on="EdCode")
deathdf = deathdf.merge(mardf, left_on="marital_status", right_on="MarCode")

# dummies for education and gender
eddummies = pd.get_dummies(deathdf["Education"])
gendummies = pd.get_dummies(deathdf["sex"])
mardummies = pd.get_dummies(deathdf["marital_status"])

deathdf = deathdf.join(eddummies)
deathdf = deathdf.join(gendummies)
deathdf = deathdf.rename(columns={"M": "Male",
                                 "F": "Female"})

deathdf = deathdf.join(mardummies)

# some weird ages that were like 999. 
# dropped anything under 150
# google says 122 is the oldest recorded so this seems safe
deathdf = deathdf[deathdf["detail_age"] < 150]

# death Dataframe split into Suicide Only Table and Non Suicide Table
SuiOnly = deathdf[deathdf["SuicideYN"] == "Suicide"]
OtherOnly = deathdf[deathdf["SuicideYN"] != "Suicide"]


## Part 3: Analysis
### Suicide Increase as a Percentage of all deaths 2005 - 2015
First initial step was to analyze how suicide was trending through the years.  I made a group by on the year of our overall death dataset.  Then using columns that were made using functions I aggregated, I found the amount of suicides by year and the amount of other deaths by year.  After plotting this out I used linear regression to find the trendline through out the years.  

In [ ]:
# Identify trend in suicide as a percentage of all deaths
# Groupby year to add up deaths and suicides
YearGrp = deathdf.groupby(["current_data_year"])

# Grouped series that are the sum of their dummies
# adding all together to find percentages
SuicideCount = YearGrp["SuicideCnt"].sum()
OtherCount = YearGrp["OtherCnt"].sum()
AllDeaths = SuicideCount + OtherCount
SuiPercent = (SuicideCount / AllDeaths)*100

# DataFrame out of grouped series
SuicideByYear = pd.DataFrame({"Suicide": SuicideCount,
                             "Other": OtherCount,
                              "Total Deaths": AllDeaths,
                             "Suicide Percent": SuiPercent})
SuicideByYear = SuicideByYear.reset_index()

# Linear regression code
year = SuicideByYear["current_data_year"]
suiper = SuicideByYear["Suicide Percent"]
sp_slope, sp_int, _, _, _ =stats.linregress(year, suiper)
sp_fit = sp_slope * year + sp_int
projyear = 2030
projected = round((sp_slope * 2025 + sp_int), 2)

# string for linear regression equation
# to be used within suicide percentage by year
textstr = "Projected to be " + str(projected) + "% by " + str(projyear) + "."

# plotting line chart with linear regression
fig, ax1 = plt.subplots(figsize=(20, 10))
ax1.set_xlim(min(year), max(year))
ax1.plot(year, suiper)
ax1.plot(year, sp_fit, "b--")
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=15,
        verticalalignment='top')
plt.tick_params(axis='both', which='major', labelsize=15)
plt.title("Suicide As a Percentage of All Deaths By Year", fontsize=20, fontweight="bold")
plt.savefig("suiPercByYear.png")

# joypy graph
#   TODO   make this more apparent in what its showing or remove it?
yearAge = SuiOnly[["current_data_year", "detail_age"]]
yearAge["current_data_year"] = yearAge["current_data_year"].astype(str)
fig, ax4 = joypy.joyplot(yearAge, by="current_data_year", figsize=(20,10),
                        ylabelsize=20, xlabelsize=20)
fig.suptitle("Suicide By Year and Age", fontsize=25, fontweight="bold")

# financial crisis caused 5000 suicides https://money.cnn.com/2013/09/18/news/economy/financial-crisis-suicide/

In [ ]:
#create education groupby to identify suicide percentage by education group
# education groupby
EdGroup = deathdf.groupby(["Education"])

# Grouped series that are the sum of their dummies
# adding all together to find percentages
EdSuiCnt = EdGroup["SuicideCnt"].sum()
EdOtherCnt = EdGroup["OtherCnt"].sum()
EdAll = EdSuiCnt + EdOtherCnt
EdSuiPercent = (EdSuiCnt / EdAll)*100

# DataFrame out of grouped series
SuicideByEducation = pd.DataFrame({"Suicide": EdSuiCnt,
                                  "Other": EdOtherCnt,
                                  "Suicide Percent":EdSuiPercent})
SuicideByEducation.plot(kind="barh", y="Suicide Percent", legend=False, figsize=(20, 10))
plt.tick_params(axis='both', which='major', labelsize=20)
plt.ylabel("")
plt.title("Suicide Percentage of Total Deaths By Education Level", fontsize=25, fontweight="bold")
plt.savefig("SuiByEd.png")

# education groupby on only suicides
EdYearGroup = SuiOnly.groupby(["current_data_year"])

# summing the dummies according to the groupby
eighth = EdYearGroup["8th grade or less"].sum()
ninth = EdYearGroup["9 - 12th grade, no diploma"].sum()
assoc = EdYearGroup["Associate degree"].sum()
bach = EdYearGroup["Bachelor’s degree"].sum()
doc = EdYearGroup["Doctorate or professional degree"].sum()
mas = EdYearGroup["Master’s degree"].sum()
unkwn = EdYearGroup["Unknown"].sum()
hs = EdYearGroup["high school graduate or GED completed"].sum()
some = EdYearGroup["some college credit, but no degree"].sum()

# dataframe of suicide only groupbys
EdYeardf = pd.DataFrame({"8th or less": eighth,
                        "9th - 12th": ninth,
                        "Associate degree": assoc,
                        "Bachelor’s degree":bach,
                        "Doctorate": doc,
                        "Master's":mas,
                        "Unknown": unkwn,
                        "High School or GED": hs,
                        "Some College":some})
EdYeardf.plot.area(figsize=(20, 10))
plt.tick_params(axis='x', which='major', labelsize=15)
plt.legend(loc=2, prop={'size': 18})
plt.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
plt.xlabel("")
plt.title("Total Suicides By Year and Education Level", fontsize=20, fontweight="bold")
plt.savefig("SuiByYearEd.png")

# education groupby on only suicides
EdYearGroupOther = OtherOnly.groupby(["current_data_year"])

# summing dummies according to groupbys
eighthother = EdYearGroupOther["8th grade or less"].sum()
ninthother = EdYearGroupOther["9 - 12th grade, no diploma"].sum()
assocother = EdYearGroupOther["Associate degree"].sum()
bachother = EdYearGroupOther["Bachelor’s degree"].sum()
docother = EdYearGroupOther["Doctorate or professional degree"].sum()
masother = EdYearGroupOther["Master’s degree"].sum()
unkwnother = EdYearGroupOther["Unknown"].sum()
hsother = EdYearGroupOther["high school graduate or GED completed"].sum()
someother = EdYearGroupOther["some college credit, but no degree"].sum()

# dataframe of year education groupby on other deaths
EdYeardfOther = pd.DataFrame({"8th or less": eighthother,
                        "9th - 12th": ninthother,
                        "Associate degree": assocother,
                        "Bachelor’s degree":bachother,
                        "Doctorate": docother,
                        "Master's":masother,
                        "Unknown": unkwnother,
                        "High School or GED": hsother,
                        "Some College":someother})

# merging the two dataframes together
# why did I do this this way??
EdYeardfAll = EdYeardf.merge(EdYeardfOther, left_index=True, right_index=True, suffixes=("","_O"))

# making percentage columns
EdYeardfAll["8thPer"] = (EdYeardfAll["8th or less"] / (EdYeardfAll["8th or less"] + EdYeardfAll["8th or less_O"]))*100
EdYeardfAll["9thPer"] = (EdYeardfAll["9th - 12th"] / (EdYeardfAll["9th - 12th"] + EdYeardfAll["9th - 12th_O"]))*100
EdYeardfAll["asscPer"] = (EdYeardfAll["Associate degree"] / (EdYeardfAll["Associate degree"] + EdYeardfAll["Associate degree_O"]))*100
EdYeardfAll["bachPer"] = (EdYeardfAll["Bachelor’s degree"] / (EdYeardfAll["Bachelor’s degree"] + EdYeardfAll["Bachelor’s degree_O"]))*100
EdYeardfAll["docPer"] = (EdYeardfAll["Doctorate"] / (EdYeardfAll["Doctorate"] + EdYeardfAll["Doctorate_O"]))*100
EdYeardfAll["masPer"] = (EdYeardfAll["Master's"] / (EdYeardfAll["Master's"] + EdYeardfAll["Master's_O"]))*100
EdYeardfAll["unkPer"] = (EdYeardfAll["Unknown"] / (EdYeardfAll["Unknown"] + EdYeardfAll["Unknown_O"]))*100
EdYeardfAll["hsPer"] = (EdYeardfAll["High School or GED"] / (EdYeardfAll["High School or GED"] + EdYeardfAll["High School or GED_O"]))*100
EdYeardfAll["somePer"] = (EdYeardfAll["Some College"] / (EdYeardfAll["Some College"] + EdYeardfAll["Some College_O"]))*100

# dropping the columns we don't need
EdYeardfAll = EdYeardfAll.drop(["8th or less", "8th or less_O",
                               "9th - 12th","9th - 12th_O",
                               "Associate degree", "Associate degree_O",
                               "Bachelor’s degree", "Bachelor’s degree_O",
                               "Doctorate", "Doctorate_O",
                               "Master's", "Master's_O",
                               "Unknown", "Unknown_O",
                               "High School or GED", "High School or GED_O",
                               "Some College", "Some College_O"], axis=1)

# renaming the remaining columns for graph aesthetics
EdYeardfAll = EdYeardfAll.rename(columns={"8thPer": "8th or less",
                                         "9thPer": "9th - 12th",
                                         "asscPer": "Associate degree",
                                         "bachPer": "Bachelor’s degree",
                                         "docPer": "Doctorate",
                                         "masPer": "Master's",
                                         "unkPer": "Unknown",
                                         "hsPer": "High School or GED",
                                         "somePer": "Some College"})
# area chart of suicide by education level
EdYeardfAll.plot.area(figsize=(20, 10))
plt.tick_params(axis='both', which='major', labelsize=15)
plt.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
plt.xlabel("")
plt.title("Suicide Percentage of Total Deaths By Year and Education Level", fontsize=20, fontweight="bold")
plt.savefig("SuiPercByYearEd.png")

# line chart of suicide by education level
colors = ["yellow", "purple"]
lineed = EdYeardfAll[["Some College", "Doctorate"]]
lineed.plot.line(figsize=(20, 10), color=colors, linewidth=7.0)
plt.tick_params(axis='both', which='major', labelsize=15)
plt.legend(loc=2, prop={'size': 18})
plt.xlabel("")
plt.title("Suicide Percentage of Total Deaths By Year and Education Level", fontsize=20, fontweight="bold")
plt.savefig("SuiPercByYearEdDocSomeCol.png")

In [ ]:

AgeGroup = SuiOnly.groupby(["detail_age"])

FireArmCnt = AgeGroup["FirearmCnt"].sum()
NonFireArmCnt = AgeGroup["NonFirearmCnt"].sum()
AllSuicides = FireArmCnt + NonFireArmCnt
FireArmPercent = (FireArmCnt/AllSuicides)*100

MthdByAge = pd.DataFrame({"Firarm":FireArmCnt,
                        "Other":NonFireArmCnt})

MthdByAge.plot(kind="bar", stacked=True, figsize=(20, 10))
plt.xlabel("")
plt.tick_params(axis='x', which='both', rotation=90)
plt.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
plt.legend(loc=2, prop={'size': 18})
plt.title("Use of Firearms Compared to Other Methods By Age", fontsize=20, fontweight="bold")
plt.savefig("SuiByFire.png")

FireArmPerdf = pd.DataFrame(FireArmPercent)
FireArmPerdf.plot(kind="line", legend=False, figsize=(20, 10), linewidth=5.0)
plt.title("Percentage Firearm Use Over Other Methods By Age", fontsize=30, fontweight="bold")
plt.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
plt.xlabel("")
plt.margins(0.01)
plt.tick_params(axis='x', which='both', bottom=False,  top=False,  labelbottom=False)
plt.savefig("SuiPerByFire.png")

In [ ]:
bins = [0, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61, 66, 71, 76, 81, 86, 91, 96, 101]

female = SuiOnly[SuiOnly["Female"] == 1]
femaleage = female["detail_age"]

male = SuiOnly[SuiOnly["Male"] == 1]
maleage = male["detail_age"]

fig, ax1 = plt.subplots(figsize=(20, 10))
plt.hist(maleage, bins=bins, alpha=0.5, label="Male")
plt.hist(femaleage, bins=bins, alpha=0.5, label="Female")
plt.title("Male and Female Suicides By Age in 5 Year Increments", fontsize=20, fontweight="bold")
plt.legend(loc=2, prop={'size': 18})
plt.tick_params(axis='x', which='major', labelsize=15)
plt.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
textstr = "Small spike for males in early twenties.  \nPossible correlation to Education Level."
plt.text(0.01, 0.83, textstr, transform=ax1.transAxes, fontsize=14,
        verticalalignment='top')
plt.savefig("SuiGenHisto.png")

# Suicide by Marital Status and Age
marAge = SuiOnly[["Marital Status", "detail_age"]]
fig, ax3 = joypy.joyplot(marAge, by="Marital Status", figsize=(20,10),
                        ylabelsize=20, xlabelsize=20)
fig.suptitle("Suicide By Marital Status and Age", fontsize=25, fontweight="bold")

# Distribution of Sex and Marital Status Over Age
agesex = SuiOnly[["Marital Status", "Male", "Female", "detail_age"]]
agesex["MaleAge"] = agesex["Male"] * agesex["detail_age"]
agesex["FemaleAge"] = agesex["Female"] * agesex["detail_age"]
agesex = agesex.replace(0, agesex.replace([0], [None]))
agesex = agesex.drop(["Male", "Female", "detail_age"], axis=1)
agesex = agesex.rename(columns={"MaleAge": "Male", "FemaleAge": "Female"})

fig, ax4 = joypy.joyplot(agesex, by="Marital Status", figsize=(20,12),
                        ylabelsize=20, xlabelsize=20, alpha=0.5, legend=True, loc=(.05, .60))
fig.suptitle("Distribution of Suicides By Sex and Marital Status Over Age", fontsize=25, fontweight="bold")
plt.savefig("SuiGenDistr.png")



marGroup = SuiOnly.groupby(["Marital Status"])

femalemar = marGroup["Female"].sum()
malemar = marGroup["Male"].sum()

marGroupdf = pd.DataFrame({"Male": malemar,
                           "Female": femalemar})


marGroupother = OtherOnly.groupby(["Marital Status"])
femalemarother = marGroupother["Female"].sum()
malemarother = marGroupother["Male"].sum()

marGroupPerc = pd.DataFrame({"Male_S": malemar,
                            "Female_S": femalemar,
                            "Male_O": malemarother,
                            "Female_O": femalemarother})
marGroupPerc["Male"] = (marGroupPerc["Male_S"] / (marGroupPerc["Male_S"] + marGroupPerc["Male_O"]))*100
marGroupPerc["Female"] = (marGroupPerc["Female_S"] / (marGroupPerc["Female_S"] + marGroupPerc["Female_O"]))*100

marGroupPerc = marGroupPerc[["Male", "Female"]]


marGroupdf.plot(kind="bar", figsize=(20, 10))
plt.tick_params(axis='x', which='major', labelsize=15)
plt.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
plt.xlabel("")
plt.xticks(rotation=0)
plt.legend(loc=2, prop={'size': 18})
plt.title("Total Suicides By Gender and Marital Status", fontsize=20, fontweight="bold")
plt.savefig("SuiByGenMar.png")

marGroupPerc.plot(kind="bar", figsize=(20, 10))
plt.tick_params(axis='both', which='major', labelsize=15)
plt.xlabel("")
plt.xticks(rotation=0)
plt.legend(loc=2, prop={'size': 18})
plt.title("Suicide as Percentage of Total Deaths By Gender and Marital Status", fontsize=20, fontweight="bold")
plt.savefig("SuiPercByGenMar.png")

# Is the rate of suicides increasing?
## Yes from the data, we can see that suicide as a percent of total deaths, that there is an increase in suicides over the past 10 years. We projected that by 2030 the suicide rate will exceed over 2% of total deaths.

# Does education play a factor in suicide?
## This is a good example where using a plt.stackplot line chart vs. a conventional plt.plot line chart is very helpful.  Using a plt.plot produces 9 separate lines, the majority of which are close together and it’s very difficult to differentiate and make determinations.  In this case, the stockplot makes a great deal of sense, showing the relative percentage of the parts vs. the whole, in this case suicide deaths by education category.  You  can see that the grey area (HS or GED) is the largest contributor to an increasing number of suicides, followed by the ‘some college’ category, and ‘bachelor’s’ and ‘9th - 12th’ are not too far behind.  However, note that this is a sub-set of our df, filtering on deaths by suicide.
## As the population grows the amount of deaths per year will also grow. As for education level, it is reported that 90% of the population 25 and older have earned a High School Degree. While only 28% over the age of 27 have college degrees. This means that the pool of "High School level education or GED" will be much larger than most, if not all, other education groups (it was by far in our data set: 400K/41% vs. 123k/13% ‘8th gr or less’). It is more appropriate to analyse this by percentage. The graph shows suicide deaths as a percentage of total deaths by year per education group.  A bit more difficult to see above, but the ‘Associates’ cat about doubled from 2005 to %2015 (1%-2%)
## Overall "Some College" is the educational group with the highest percentage suicides of total deaths.  Counter-intuitively, the highest suicide rates were all higher education levels:  Some college: 2.26%, Associates: 1.95%, Doctorate: 1.93%, Bachelor's: 1.92%, Master's: 1.72%. In line with this relationship, the lowest level of education completed expressed in numbers and percentage of whole, was 8th grade or less.

# Do most suicides involve firearms?
## Using the "Cause of Death" code in our dataset we were able to analyse a change in the preferred method overtime. Firearms were the largest proportion of suicide method in our dataset, so they were contrasted against everything else. As one gets older or less after the age of 50, people tend to commit suicide less as a whole, but at the same time as more people approach the age of 50 and over, it is observed that they tend to use firearms as a preferred method.
## Firearms account for 42% of suicides under the age of 50.
## After the age of 50, usage increases to 58.4%.


# Is suicide more prevalent in one age group over others?
## From the data seen, it was apperent that typically people from the age of 15-24 were at the highest risk to commit suicide, not considering other factors just age. We did consider outside factors from our data such as income level, the effect of social media, the possibility of bullying. 

# Does Suicide differ depending on Sex?
## Even without showing the number of suicides it's easy to see that suicides of males is double that of females; for instance for the age range 10-15 female suicides are 45% of the male total. What’s interesting to point out male suicide skyrockets 241% from 11-15 age group to 16-20 age group and jumps another 28.61% for 21-25, but by comparison, female suicides only jumps 111.24% from 11-15 age group to 16-20, which again is around half that of males. Both males and females seem to have a similar pattern; except for men in their early twenties, this may have a possible correlation to education level.
# Marital status & suicide: A distinguishable relationship?
## Adding a third demographic marital status, we see the two groups that don't have a normal distribution are never married and widowed; this doesn’t come as a huge surprise. We can safely assume the two groups follow suit with their typical age distribution. Usually there are more never married individuals in their 20-30s then 60s and the reverse for widowed. It would be interesting to do further investigation to see how long the individual was a widow before passing away. The other groups have a fairly normal distribution and the peak seems to fall around the mid-life crisis time frame. Recalling from suicides by sex and age, we saw there was a spike for the number of men in their 20s; it seems the majority of these men were never married. Further investigation would be needed to do a correlation, but it’s safe to assume there is some sort of relationship.
## When it comes to suicide there are rate differences depending on demographic characteristics such as age, gender, marital status, education, and numerous others. Nonetheless, suicide occurs in all demographic groups.